# Pyrope: Gemini-Driven Autonomous Vector Database

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/takurot/Pyrope/blob/main/example/pyrope_colab_demo.ipynb)

Pyrope is a self-driving vector database that integrates **Microsoft Garnet** with **Google Gemini**. 
In this notebook, we will:
1. Setup the Pyrope environment (.NET + Python).
2. Start the AI Sidecar (Gemini-powered controller).
3. Start the Garnet Server.
4. Perform vector search and see Gemini's autonomous policy in action.

## 1. Setup Environment

In [ ]:
# Install .NET SDK (Required for Garnet)
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0
import os
os.environ["PATH"] = f"{os.environ['HOME']}/.dotnet:" + os.environ["PATH"]

# Clone the repository
!git clone https://github.com/takurot/Pyrope.git
%cd Pyrope

In [ ]:
# Install Python dependencies for AI Sidecar
!pip install grpcio grpcio-tools google-generativeai psutil redis

## 2. Configure Gemini API Key
Set your Gemini API key from [Google AI Studio](https://aistudio.google.com/).

In [ ]:
import getpass
os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Gemini API Key: ")
os.environ["LLM_POLICY_ENABLED"] = "true"

## 3. Launch Services

In [ ]:
import subprocess
import time

print("Starting AI Sidecar...")
sidecar_process = subprocess.Popen(
    ["python", "src/Pyrope.AISidecar/server.py"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(5)

print("Starting Garnet Server (Pyrope)...")
server_process = subprocess.Popen(
    ["dotnet", "run", "--project", "src/Pyrope.GarnetServer", "--configuration", "Release", "--", "--port", "6379"],
    env={**os.environ, "Auth__Enabled": "false", "Sidecar__Endpoint": "http://127.0.0.1:50051", "Sidecar__MetricsIntervalSeconds": "2"},
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

time.sleep(10)
print("Services started!")

## 4. Run Vector Operations
We use standard Redis client to interact with Pyrope's `VEC.*` commands.

In [ ]:
import redis
import json
import random
import struct

r = redis.Redis(host='localhost', port=6379, decode_responses=False)

TENANT = "colab_user"
INDEX = "demo_index"
DIM = 32

def float_list_to_binary(floats):
    return struct.pack(f"{len(floats)}f", *floats)

print("Registering index...")
# Note: In a real scenario, use HTTP API to create index, but here we just add vectors

print("Adding vectors...")
for i in range(100):
    vec = [random.random() for _ in range(DIM)]
    # VEC.ADD <tenant> <index> <id> VECTOR <blob> META <json>
    r.execute_command("VEC.ADD", TENANT, INDEX, f"v{i}", "VECTOR", float_list_to_binary(vec), "META", json.dumps({"label": i}))

print("Performing search...")
query = [random.random() for _ in range(DIM)]
results = r.execute_command("VEC.SEARCH", TENANT, INDEX, "VECTOR", float_list_to_binary(query), "TOPK", 5)
print(f"Search Results: {results}")

## 5. Check AI Sidecar Logs
Let's see how Gemini is responding to the metrics.

In [ ]:
# Read the latest lines from sidecar output
print("--- Sidecar Logs ---")
for line in sidecar_process.stdout:
    if "Policy" in line:
        print(line.strip())
    if sidecar_process.poll() is not None:
        break

## Cleanup

In [ ]:
server_process.terminate()
sidecar_process.terminate()